In [88]:
import torch
import scipy.io as io
from __future__ import print_function, division
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

import torch.nn as nn
import torch.nn.functional as F

In [89]:
headers='hw5_data_code/deap_features.mat'
dataHeaders= io.loadmat(headers)

dataFile = 'hw5_data_code/results_DEAP.mat'
#dataFile= io.loadmat(dataFile)
#fname=data['label','PID','image','tumorBorder','tumorMask'] 

In [90]:
features_frame = np.zeros(shape=(1280, 6))
print(features_frame)

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


In [91]:
class FeaturesDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, data_file, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.features_frame = io.loadmat(data_file)#pd.read_csv(csv_file)
        self.features_frame = np.zeros(shape=(1280, 6))
        i = 0
        #print(dataHeaders.shape)
        #for line in dataHeaders:
        subjects = dataHeaders['features']
        # read each feature set for each video for this subject into the data array
        for subject in subjects:
            for feature in subject:
                # get mean of means for EMG features
                mean_emg = np.mean(feature['EMG_feats'], axis=0)
                mean_emg = mean_emg[0]
                #print(mean_emg)
                self.features_frame[i][0] = mean_emg
                # set GSR mean
                #print(feature['GSR_feats'])
                self.features_frame[i][1] = feature['GSR_feats'][0][3]
                # set RES mean
                self.features_frame[i][2] = feature['RES_feats'][0][0]
                # set BVP mean
                #print("BVP")
                #print(feature['BVP_feats'])
                self.features_frame[i][3] = feature['BVP_feats'][0][0]
                self.features_frame[i]
                #print(feature['feedback']['felt_arousal'])
                #print(feature['feedback']['felt_arousal'])
                self.features_frame[i][4] = feature['feedback']['felt_arousal']
                self.features_frame[i][5] = feature['feedback']['felt_valence']
                i += 1
        # convert to pytorch tensor
        self.features_frame = torch.from_numpy(self.features_frame)
        #self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.features_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        #img_name = os.path.join(self.root_dir,
        #                        self.features_frame.iloc[idx, 0])
        #image = io.imread(img_name)
        feature_set = self.features_frame[idx][0:4]
        arousal = self.features_frame[idx][4]
        valence = self.features_frame[idx][5]
        #landmarks = np.array([landmarks])
        #landmarks = landmarks.astype('float').reshape(-1, 2)
        #print(torch.tensor([arousal]))
        sample = {'features': feature_set, 'arousal':torch.tensor([arousal]), 'valence': valence}

        if self.transform:
            sample = self.transform(sample)

        return sample

In [92]:
face_dataset = FeaturesDataset(dataFile)

In [49]:
#features_frame = io.loadmat(dataFile)#pd.read_csv(csv_file)
#features_frame = np.zeros(shape=(1280, 4))


In [93]:
#our class must extend nn.Module
class MyClassifier(nn.Module):
    def __init__(self):
        super(MyClassifier,self).__init__()
        #Our network consists of 3 layers. 1 input, 1 hidden and 1 output layer
        #This applies Linear transformation to input data. 
        self.fc1 = nn.Linear(4,4)
        
        #This applies linear transformation to produce output data
        self.fc2 = nn.Linear(4,1)
        
    #This must be implemented
    def forward(self,x):
        #Output of the first layer
        x = self.fc1(x)
        #Activation function is Relu. Feel free to experiment with this
        x = F.tanh(x)
        #This produces output
        x = self.fc2(x)
        return x
        
    #This function takes an input and predicts the class, (0 or 1)        
    def predict(self,x):
        #Apply softmax to output. 
        #pred = F.softmax(self.forward(x))
        pred = self.forward(x)#F.softmax(self.forward(x))
        ans = []
        #Pick the class with maximum weight
        for t in pred:
            if t[0]>t[1]:
                ans.append(0)
            else:
                ans.append(1)
        return torch.tensor(ans)

In [94]:
#Initialize the model        
model = MyClassifier()
#Define loss criterion
criterion = torch.nn.L1Loss()#nn.CrossEntropyLoss()
#Define the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [95]:
face_dataset = FeaturesDataset(dataFile)
trainloader = torch.utils.data.DataLoader(face_dataset, batch_size=64, shuffle=True)

# Define the loss
#criterion = nn.NLLLoss()
criterion = nn.L1Loss()
# Optimizers require the parameters to optimize and a learning rate
#optimizer = optim.SGD(model.parameters(), lr=0.003)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
epochs = 3000
model = model.float()
i = 0
for e in range(epochs):
    running_loss = 0
    for samples in trainloader:
        #print(sample['features'])
        #print(arousal)
        #print(felt)
        # Flatten MNIST images into a 784 long vector
        #images = images.view(images.shape[0], -1)
        
        # Training pass
        #optimizer.zero_grad()
        
        #output = model(images)
        #loss = criterion(output, labels)
        #loss.backward()
        #optimizer.step()
        x = samples['features']
        x = x.float()
        y_pred = model.forward(x)
        #print(np.array(sample['arousal']))
        #print(torch.tensor([arousal]))
        #print(y_pred)
        loss = criterion(y_pred, samples['arousal'])
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        i += 1
        if i % 1000 == 0:
            print(f"Training loss: {running_loss/len(trainloader)}")

Training loss: 2.5491184255182744


KeyboardInterrupt: 

In [96]:
def calculateF1Score(correctHigh, falsePositives, totalHigh):
    precision = correctHigh / (correctHigh + falsePositives)
    # r is the number of correct positive results divided by the number of all relevant samples (all samples that should have been identified as positive)
    recall = correctHigh / totalHigh
    f1 = 2 * (precision * recall) / (precision + recall)
    return f1

In [97]:
def trainNet(model, trainloader, epochs, label):
    # Define the loss
    #criterion = nn.NLLLoss()
    criterion = nn.L1Loss()
    # Optimizers require the parameters to optimize and a learning rate
    #optimizer = optim.SGD(model.parameters(), lr=0.003)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    #epochs = 3000
    model = model.float()
    i = 0
    for e in range(epochs):
        running_loss = 0
        for samples in trainloader:
            #print(sample['features'])
            #print(arousal)
            #print(felt)
            # Flatten MNIST images into a 784 long vector
            #images = images.view(images.shape[0], -1)

            # Training pass
            #optimizer.zero_grad()

            #output = model(images)
            #loss = criterion(output, labels)
            #loss.backward()
            #optimizer.step()
            x = samples['features']
            x = x.float()
            y_pred = model.forward(x)
            #print(np.array(sample['arousal']))
            #print(torch.tensor([arousal]))
            #print(y_pred)
            if label == 'arousal':
                loss = criterion(y_pred, samples['arousal'])
            elif label == 'valence':
                loss = criterion(y_pred, samples['valence'])
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            i += 1
            #if i % 1000 == 0:
    print(f"Training loss: {running_loss/len(trainloader)}")

In [103]:
def testNet(model, trainloader, label):
    # Define the loss
    #criterion = nn.NLLLoss()
    criterion = nn.L1Loss()
    
    # Define the loss
    #criterion = nn.NLLLoss()
    criterion = nn.L1Loss()
    # Optimizers require the parameters to optimize and a learning rate
    #optimizer = optim.SGD(model.parameters(), lr=0.003)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    epochs = 30
    model = model.float()
    # how many were correctly guessed as high arousal
    correctHigh = 0
    # how many were actually high
    totalHigh = 0
    # how many were guessed as high
    falsePositives = 0
    # how many were correctly guessed as low arousal
    correctLow = 0
    # how many were actually low
    totalLow = 0
    # how many were guessed as low
    falseNegatives = 0
    for samples in trainloader:
        #print(sample['features'])
        #print(arousal)
        #print(felt)
        # Flatten MNIST images into a 784 long vector
        #images = images.view(images.shape[0], -1)

        # Training pass
        #optimizer.zero_grad()

        #output = model(images)
        #loss = criterion(output, labels)
        #loss.backward()
        #optimizer.step()
        x = samples['features']
        x = x.float()
        y_pred = model.forward(x)
        #print("y %f x %f" %(y_pred, samples['arousal']))
        print(samples['valence'])
        if label == 'valence':
            #print(samples['arousal'])
            if samples['valence'] > 5:
                totalHigh += 1
                if y_pred > 5:
                    correctHigh += 1
            elif y_pred > 5:
                falsePositives += 1
            if samples['valence'] < 5:
                totalLow += 1
                if y_pred < 5:
                    correctLow += 1
            elif y_pred < 5:
                falseNegatives += 1
        else:
            #print(samples['arousal'])
            if samples['arousal'] > 5:
                totalHigh += 1
                if y_pred > 5:
                    correctHigh += 1
            elif y_pred > 5:
                falsePositives += 1
            if samples['arousal'] < 5:
                totalLow += 1
                if y_pred < 5:
                    correctLow += 1
            elif y_pred < 5:
                falseNegatives += 1
    print("correct high %d total high %d percentage correct %f false positives %d" % (correctHigh, totalHigh, ((correctHigh / totalHigh) * 100), falsePositives))
    print("correct negative %d total low %d percentage correct %f false negatives %d" % (correctLow, totalLow, ((correctLow / totalLow) * 100), falseNegatives))

    print("f1 arousal %f" %(calculateF1Score(correctHigh, falsePositives, totalHigh)))

In [104]:
FeaturesDataset(dataFile)
trainloader = torch.utils.data.DataLoader(face_dataset, batch_size=64, shuffle=True)
trainNet(model, trainloader, 1000, 'valence')
testNet(model, trainloader, 'valence')
# need to beat 
#trainNet(model, trainloader, 5000, 'arousal')
#testNet(model, trainloader, 'arousal')

Training loss: 3.8358828237876272
tensor([1.9900, 5.0400, 6.9000, 1.0400, 1.9600, 1.0000, 6.0400, 4.0400, 1.0100,
        3.9900, 4.9500, 6.9400, 4.6200, 6.0300, 4.0500, 6.3600, 7.0100, 6.7200,
        4.9900, 7.9700, 7.0300, 6.9900, 6.1400, 2.0300, 7.4400, 3.9100, 8.0600,
        1.0000, 5.1500, 5.0400, 6.8700, 7.0400, 4.1200, 3.1200, 5.0400, 9.0000,
        6.0000, 6.0300, 1.2100, 2.5100, 3.4900, 8.9000, 4.9600, 4.0400, 2.0800,
        8.0800, 5.1500, 7.0900, 4.0600, 6.8200, 4.1800, 7.5600, 6.8100, 3.1900,
        6.1300, 7.6900, 4.5300, 4.9700, 6.9100, 4.0900, 2.9900, 6.0600, 5.9700,
        3.1000], dtype=torch.float64)


RuntimeError: bool value of Tensor with more than one value is ambiguous

In [87]:
testNet(model, trainloader, 'valence')

correct high 26 total high 708 percentage correct 3.672316 false positives 15
correct negative 541 total low 556 percentage correct 97.302158 false negatives 698
f1 arousal 0.069426


In [ ]:
def calculateF1Score(correctHigh, falsePositives, totalHigh):
    precision = correctHigh / (correctHigh + falsePositives)
    # r is the number of correct positive results divided by the number of all relevant samples (all samples that should have been identified as positive)
    recall = correctHigh / totalHigh
    f1 = 2 * (precision * recall) / (precision + recall)
    return f1